In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

data = pd.read_csv('democracy.csv')

In [89]:
modern_data = data[data['Year'] > 2016]
modern_data.rename(columns={'electdem_vdem_owid': 'Index'}, inplace=True)
modern_data.drop(['electdem_vdem_high_owid','electdem_vdem_low_owid'], axis=1, inplace=True)
non_modern_data = data[data['Year'] <= 2016]
non_modern_data.rename(columns={'electdem_vdem_owid': 'Index'}, inplace=True)
non_modern_data.drop(['electdem_vdem_high_owid','electdem_vdem_low_owid'], axis=1, inplace=True)


C:\Users\jstou\AppData\Local\Temp\ipykernel_1512\1065609558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_data.rename(columns={'electdem_vdem_owid': 'Index'}, inplace=True)
C:\Users\jstou\AppData\Local\Temp\ipykernel_1512\1065609558.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_data.drop(['electdem_vdem_high_owid','electdem_vdem_low_owid'], axis=1, inplace=True)
C:\Users\jstou\AppData\Local\Temp\ipykernel_1512\1065609558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [90]:
continents = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'World']
for continent in continents:
    if continent in modern_data.columns:
        modern_data.drop(continent, axis=1, inplace=True)

In [91]:
all_countries = modern_data['Entity'].unique()
predictions_country = {'Curve': [], 'Country': []}

for country in all_countries:
    country_data = modern_data[modern_data['Entity'] == country]
    X = country_data['Year'].values.reshape(-1, 1)
    y = country_data['Index'].values.reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, y)
    slope = model.coef_[0][0]
    intercept = model.intercept_[0]
    curve = [slope, intercept]
    predictions_country['Curve'].append(curve)
    predictions_country['Country'].append(country)

predictions_country = pd.DataFrame(predictions_country)
predictions_country.head()


,Curve,Country
0,"[-0.05557142857142856, 112.50266666666666]",Afghanistan
1,"[-0.008827041142857149, 18.241379231333344]",Africa
2,"[-0.0057142857142857195, 12.035000000000009]",Albania
3,"[-0.006228571428571419, 12.868933333333315]",Algeria
4,"[0.00048571428571428214, -0.632733333333326]",Angola


In [92]:
points = []
country = []
code = []

predictions_code = modern_data[modern_data['Year'] == 2020]

x = np.arange(2023, 2031, 1)
for i in range(len(predictions_country['Curve'])):
    for j in x:
        value = predictions_country['Curve'][i][0] * j + predictions_country['Curve'][i][1]
        points.append(value)
        country.append(predictions_country['Country'][i])
        code.append(predictions_code['Code'].iloc[i])

all_years = []    
for i in range(len(all_countries)):
    for j in x:
        all_years.append(j)
        

predictions = pd.DataFrame({'Entity': country, 'Code': code, 'Year': all_years, 'Index': points})
predictions = predictions.sort_values(by=['Entity', 'Year'])
predictions
        

,Entity,Code,Year,Index
0,Afghanistan,AFG,2023,0.081667
1,Afghanistan,AFG,2024,0.026095
2,Afghanistan,AFG,2025,-0.029476
3,Afghanistan,AFG,2026,-0.085048
4,Afghanistan,AFG,2027,-0.140619
...,...,...,...,...
1483,Zimbabwe,ZWE,2026,0.276581
1484,Zimbabwe,ZWE,2027,0.273952
1485,Zimbabwe,ZWE,2028,0.271324
1486,Zimbabwe,ZWE,2029,0.268695


In [93]:
continents = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']
for continent in continents:
    if continent in predictions['Entity']:
        predictions = predictions.drop(continent)
filtered = predictions[predictions['Entity'] != 'World']

filtered['Index'] = round(filtered['Index'], 3)
filtered

C:\Users\jstou\AppData\Local\Temp\ipykernel_1512\527638263.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Index'] = round(filtered['Index'], 3)


,Entity,Code,Year,Index
0,Afghanistan,AFG,2023,0.082
1,Afghanistan,AFG,2024,0.026
2,Afghanistan,AFG,2025,-0.029
3,Afghanistan,AFG,2026,-0.085
4,Afghanistan,AFG,2027,-0.141
...,...,...,...,...
1483,Zimbabwe,ZWE,2026,0.277
1484,Zimbabwe,ZWE,2027,0.274
1485,Zimbabwe,ZWE,2028,0.271
1486,Zimbabwe,ZWE,2029,0.269


In [94]:
predict = pd.concat([non_modern_data, modern_data, filtered])
predict.sort_values(by=['Entity', 'Year'], inplace=True)
predict = predict[round(predict['Index'], 3) <= 1.00]
predict.to_csv('predictions.csv', index=True)